<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Preparation</a></span><ul class="toc-item"><li><span><a href="#Hillstrom" data-toc-modified-id="Hillstrom-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hillstrom</a></span></li><li><span><a href="#Mayo-PBC" data-toc-modified-id="Mayo-PBC-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Mayo PBC</a></span></li><li><span><a href="#CMF-Microfinance" data-toc-modified-id="CMF-Microfinance-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CMF Microfinance</a></span></li></ul></li><li><span><a href="#Iterative-Modeling" data-toc-modified-id="Iterative-Modeling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Iterative Modeling</a></span></li><li><span><a href="#Evaluation-and-Variance-Table" data-toc-modified-id="Evaluation-and-Variance-Table-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Evaluation and Variance Table</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from causeinfer import utils
from causeinfer.data import hillstrom, mayo_pbc, cmf_micro
from causeinfer.standard_algorithms.two_model import TwoModel
from causeinfer.standard_algorithms.interaction_term import InteractionTerm
from causeinfer.standard_algorithms.binary_transformation import BinaryTransformation
from causeinfer.standard_algorithms.quaternary_transformation import (
    QuaternaryTransformation,
)
from causeinfer.evaluation import qini_score, auuc_score
from causeinfer.evaluation import plot_cum_effect, plot_cum_gain, plot_qini
from causeinfer.evaluation import plot_batch_responses, signal_to_noise
from causeinfer.evaluation import iterate_model, eval_table

pd.set_option("display.max_rows", 16)
pd.set_option("display.max_columns", None)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

# Load Data

In [2]:
hillstrom.download_hillstrom()
mayo_pbc.download_mayo_pbc()

The dataset already exists at /Users/andrewtavis/Documents/learning/coding/causeinfer/examples/datasets/hillstrom.csv
The dataset already exists at /Users/andrewtavis/Documents/learning/coding/causeinfer/examples/datasets/mayo_pbc.text


In [3]:
data_hillstrom = hillstrom.load_hillstrom(
    file_path="datasets/hillstrom.csv", format_covariates=True, normalize=True
)
data_mayo_pbc = mayo_pbc.load_mayo_pbc(
    file_path="datasets/mayo_pbc.text", format_covariates=True, normalize=True
)
data_cmf_micro = cmf_micro.load_cmf_micro(
    file_path="datasets/cmf_micro", format_covariates=True, normalize=True
)

/Users/andrewtavis/opt/anaconda3/envs/causeinfer-dev/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


# Data Preparation

## Hillstrom

In [4]:
# Covariates, treatments and responses are loaded separately
X_hillstrom = data_hillstrom["features"]

y_hillstrom = data_hillstrom[
    "response_visit"
]  # response_visit, response_spend or response_conversion

# 1 is men's campaign, 2 is women's, and 0 is control
w_hillstrom = data_hillstrom["treatment"]

In [5]:
# Counts for treatment
control_indexes_hillstrom = [i for i, e in enumerate(w_hillstrom) if e == 0]
mens_indexes = [i for i, e in enumerate(w_hillstrom) if e == 1]
womens_indexes = [i for i, e in enumerate(w_hillstrom) if e == 2]

womens_mens_indexes = womens_indexes + mens_indexes

print(len(control_indexes_hillstrom))
print(len(mens_indexes))
print(len(womens_indexes))
print(len(womens_mens_indexes))

21306
21307
21387
42694


In [6]:
X_control_hillstrom = X_hillstrom[control_indexes_hillstrom]
y_control_hillstrom = y_hillstrom[control_indexes_hillstrom]
w_control_hillstrom = w_hillstrom[control_indexes_hillstrom]

X_women = X_hillstrom[womens_indexes]
y_women = y_hillstrom[womens_indexes]
w_women = w_hillstrom[womens_indexes]

In [7]:
# Change 2s to 1s in women's campaign
w_women = [1 for i in w_women if i == 2]
w_women[:5]

[1, 1, 1, 1, 1]

In [8]:
# Over-sampling of control
X_os_hillstrom, y_os_hillstrom, w_os_hillstrom = utils.over_sample(
    X_1=X_control_hillstrom,
    y_1=y_control_hillstrom,
    w_1=w_control_hillstrom,
    sample_2_size=len(X_women),
    shuffle=True,
)


    Old Covariates shape  : (21306, 18)
    Old responses shape   : (21306,)
    Old treatments shape  : (21306,)
    New covariates shape  : (21387, 18)
    New responses shape   : (21387,)
    New treatments shape  : (21387,)
    Matched sample length :  21387
                        


In [9]:
X_split_hillstrom = np.append(X_os_hillstrom, X_women, axis=0)
y_split_hillstrom = np.append(y_os_hillstrom, y_women, axis=0)
w_split_hillstrom = np.append(w_os_hillstrom, w_women, axis=0)

X_split_hillstrom.shape, y_split_hillstrom.shape, w_split_hillstrom.shape  # Should all be equal in the first dimension

((42774, 18), (42774,), (42774,))

In [10]:
(
    X_train_hillstrom,
    X_test_hillstrom,
    y_train_hillstrom,
    y_test_hillstrom,
    w_train_hillstrom,
    w_test_hillstrom,
) = utils.train_test_split(
    X_split_hillstrom,
    y_split_hillstrom,
    w_split_hillstrom,
    percent_train=0.7,
    random_state=42,
    maintain_proportions=True,
)
X_train_hillstrom.shape, X_test_hillstrom.shape, y_train_hillstrom.shape, y_test_hillstrom.shape, w_train_hillstrom.shape, w_test_hillstrom.shape

((29940, 18), (12834, 18), (29940,), (12834,), (29940,), (12834,))

In [11]:
print(np.array(np.unique(w_train_hillstrom, return_counts=True)).T)
print(np.array(np.unique(w_test_hillstrom, return_counts=True)).T)

[[    0 14970]
 [    1 14970]]
[[   0 6417]
 [   1 6417]]


## Mayo PBC

In [12]:
# Covariates, treatments and responses are loaded separately
X_mayo = data_mayo_pbc["features"]

# 0 is the patient is alive, 1 is a liver transplant, 2 is deceased
y_mayo = data_mayo_pbc["response"]

w_mayo = data_mayo_pbc["treatment"]

In [13]:
# Counts for response
alive_indexes = [i for i, e in enumerate(y_mayo) if e == 0]
transplant_indexes = [i for i, e in enumerate(y_mayo) if e == 1]
deceased_indexes = [i for i, e in enumerate(y_mayo) if e == 2]

transplant_deceased_indexes = transplant_indexes + deceased_indexes

print(len(alive_indexes))
print(len(transplant_indexes))
print(len(deceased_indexes))
print(len(transplant_deceased_indexes))

168
19
125
144


In [14]:
y_mayo = np.array(
    [1 if i in transplant_deceased_indexes else 0 for i, e in enumerate(y_mayo)]
)

In [15]:
# Counts for treatment
control_indexes_mayo = [i for i, e in enumerate(w_mayo) if e == 0]
treatment_indexes_mayo = [i for i, e in enumerate(w_mayo) if e == 1]

print(len(control_indexes_mayo))
print(len(treatment_indexes_mayo))

154
158


In [16]:
X_control_mayo = X_mayo[control_indexes_mayo]
y_control_mayo = y_mayo[control_indexes_mayo]
w_control_mayo = w_mayo[control_indexes_mayo]

X_treatment_mayo = X_mayo[treatment_indexes_mayo]
y_treatment_mayo = y_mayo[treatment_indexes_mayo]
w_treatment_mayo = w_mayo[treatment_indexes_mayo]

In [17]:
# Over-sampling of control
X_os_mayo, y_os_mayo, w_os_mayo = utils.over_sample(
    X_1=X_control_mayo,
    y_1=y_control_mayo,
    w_1=w_control_mayo,
    sample_2_size=len(X_treatment_mayo),
    shuffle=True,
)


    Old Covariates shape  : (154, 22)
    Old responses shape   : (154,)
    Old treatments shape  : (154,)
    New covariates shape  : (158, 22)
    New responses shape   : (158,)
    New treatments shape  : (158,)
    Matched sample length :  158
                        


In [18]:
X_split_mayo = np.append(X_os_mayo, X_treatment_mayo, axis=0)
y_split_mayo = np.append(y_os_mayo, y_treatment_mayo, axis=0)
w_split_mayo = np.append(w_os_mayo, w_treatment_mayo, axis=0)

X_split_mayo.shape, y_split_mayo.shape, w_split_mayo.shape  # Should all be equal in the first dimension

((316, 22), (316,), (316,))

In [19]:
(
    X_train_mayo,
    X_test_mayo,
    y_train_mayo,
    y_test_mayo,
    w_train_mayo,
    w_test_mayo,
) = utils.train_test_split(
    X_split_mayo,
    y_split_mayo,
    w_split_mayo,
    percent_train=0.7,
    random_state=42,
    maintain_proportions=True,
)

X_train_mayo.shape, X_test_mayo.shape, y_train_mayo.shape, y_test_mayo.shape, w_train_mayo.shape, w_test_mayo.shape

((220, 22), (96, 22), (220,), (96,), (220,), (96,))

## CMF Microfinance

In [20]:
X_cmf = data_cmf_micro["features"]

y_cmf = data_cmf_micro["response_biz_index"]  # response_biz_index or response_women_emp

w_cmf = data_cmf_micro["treatment"]

In [21]:
# Counts for treatment
control_indexes = [i for i, e in enumerate(w_cmf) if e == 0]
treatment_indexes = [i for i, e in enumerate(w_cmf) if e == 1]

print(len(control_indexes))
print(len(treatment_indexes))

2576
2752


In [22]:
X_control_cmf = X_cmf[control_indexes]
y_control_cmf = y_cmf[control_indexes]
w_control_cmf = w_cmf[control_indexes]

X_treatment_cmf = X_cmf[treatment_indexes]
y_treatment_cmf = y_cmf[treatment_indexes]
w_treatment_cmf = w_cmf[treatment_indexes]

In [23]:
# Over-sampling of control
X_os_cmf, y_os_cmf, w_os_cmf = utils.over_sample(
    X_1=X_control_cmf,
    y_1=y_control_cmf,
    w_1=w_control_cmf,
    sample_2_size=len(X_treatment_cmf),
    shuffle=True,
)


    Old Covariates shape  : (2576, 160)
    Old responses shape   : (2576,)
    Old treatments shape  : (2576,)
    New covariates shape  : (2752, 160)
    New responses shape   : (2752,)
    New treatments shape  : (2752,)
    Matched sample length :  2752
                        


In [24]:
X_split_cmf = np.append(X_os_cmf, X_treatment_cmf, axis=0)
y_split_cmf = np.append(y_os_cmf, y_treatment_cmf, axis=0)
w_split_cmf = np.append(w_os_cmf, w_treatment_cmf, axis=0)

X_split_cmf.shape, y_split_cmf.shape, w_split_cmf.shape  # Should all be equal in the first dimension

((5504, 160), (5504,), (5504,))

In [25]:
(
    X_train_cmf,
    X_test_cmf,
    y_train_cmf,
    y_test_cmf,
    w_train_cmf,
    w_test_cmf,
) = utils.train_test_split(
    X_split_cmf,
    y_split_cmf,
    w_split_cmf,
    percent_train=0.7,
    random_state=42,
    maintain_proportions=True,
)

X_train_cmf.shape, X_test_cmf.shape, y_train_cmf.shape, y_test_cmf.shape, w_train_cmf.shape, w_test_cmf.shape

((3852, 160), (1652, 160), (3852,), (1652,), (3852,), (1652,))

# Iterative Modeling

In [26]:
dataset_keys = {
    "Hillstrom": {
        "X_train": X_train_hillstrom,
        "y_train": y_train_hillstrom,
        "w_train": w_train_hillstrom,
        "X_test": X_test_hillstrom,
        "y_test": y_test_hillstrom,
        "w_test": w_test_hillstrom,
    },
    "Mayo PBC": {
        "X_train": X_train_mayo,
        "y_train": y_train_mayo,
        "w_train": w_train_mayo,
        "X_test": X_test_mayo,
        "y_test": y_test_mayo,
        "w_test": w_test_mayo,
    },
    "CMF Microfinance": {
        "X_train": X_train_cmf,
        "y_train": y_train_cmf,
        "w_train": w_train_cmf,
        "X_test": X_test_cmf,
        "y_test": y_test_cmf,
        "w_test": w_test_cmf,
    },
}

Scikit-Learn models to use:

- RandomForestClassifier() for Hillstrom and Mayo PBC
- RandomForestRegressor() for CMF Microfinance

In [27]:
n = 200
model_eval_dict = {}
model_eval_dict["Hillstrom"] = {}
model_eval_dict["Mayo PBC"] = {}
model_eval_dict["CMF Microfinance"] = {}
model_eval_dict

{'Hillstrom': {}, 'Mayo PBC': {}, 'CMF Microfinance': {}}

In [28]:
for dataset in dataset_keys.keys():
    if dataset in ["Hillstrom", "Mayo PBC"]:  # predict_proba
        tm_class = TwoModel(
            treatment_model=RandomForestClassifier(
                n_estimators=200, criterion="gini", bootstrap=True
            ),
            control_model=RandomForestClassifier(
                n_estimators=200, criterion="gini", bootstrap=True
            ),
        )

        it_class = InteractionTerm(
            model=RandomForestClassifier(
                n_estimators=200, criterion="gini", bootstrap=True
            )
        )

        bt_class = BinaryTransformation(
            model=RandomForestClassifier(
                n_estimators=200, criterion="gini", bootstrap=True
            ),
            regularize=True,
        )

        qt_class = QuaternaryTransformation(
            model=RandomForestClassifier(
                n_estimators=200, criterion="gini", bootstrap=True
            ),
            regularize=True,
        )
        print("---{} Iterations---".format(dataset))
        for model in [tm_class, it_class, bt_class, qt_class]:
            (
                avg_preds,
                all_preds,
                avg_eval,
                eval_variance,
                eval_sd,
                all_evals,
            ) = iterate_model(
                model=model,
                X_train=dataset_keys[dataset]["X_train"],
                y_train=dataset_keys[dataset]["y_train"],
                w_train=dataset_keys[dataset]["w_train"],
                X_test=dataset_keys[dataset]["X_test"],
                y_test=dataset_keys[dataset]["y_test"],
                w_test=dataset_keys[dataset]["w_test"],
                tau_test=None,
                n=n,
                pred_type="predict_proba",
                eval_type="qini",
                normalize_eval=False,
                verbose=False,  # Progress bar
            )

            model_eval_dict[dataset].update(
                {
                    str(model)
                    .split(".")[-1]
                    .split(" ")[0]: {
                        "avg_preds": avg_preds,
                        "all_preds": all_preds,
                        "avg_eval": avg_eval,
                        "eval_variance": eval_variance,
                        "eval_sd": eval_sd,
                        "all_evals": all_evals,
                    }
                }
            )

    else:  # predict
        tm_reg = TwoModel(
            treatment_model=RandomForestRegressor(
                n_estimators=200, criterion="mse", bootstrap=True
            ),
            control_model=RandomForestRegressor(
                n_estimators=200, criterion="mse", bootstrap=True
            ),
        )

        it_reg = InteractionTerm(
            model=RandomForestRegressor(
                n_estimators=200, criterion="mse", bootstrap=True
            )
        )
        print("---{} Iterations---".format(dataset))
        for model in [tm_reg, it_reg]:
            (
                avg_preds,
                all_preds,
                avg_eval,
                eval_variance,
                eval_sd,
                all_evals,
            ) = iterate_model(
                model=model,
                X_train=dataset_keys[dataset]["X_train"],
                y_train=dataset_keys[dataset]["y_train"],
                w_train=dataset_keys[dataset]["w_train"],
                X_test=dataset_keys[dataset]["X_test"],
                y_test=dataset_keys[dataset]["y_test"],
                w_test=dataset_keys[dataset]["w_test"],
                tau_test=None,
                n=n,
                pred_type="predict",
                eval_type="qini",
                normalize_eval=False,
                verbose=False,  # Progress bar
            )

            model_eval_dict[dataset].update(
                {
                    str(model)
                    .split(".")[-1]
                    .split(" ")[0]: {
                        "avg_preds": avg_preds,
                        "all_preds": all_preds,
                        "avg_eval": avg_eval,
                        "eval_variance": eval_variance,
                        "eval_sd": eval_sd,
                        "all_evals": all_evals,
                    }
                }
            )

---Hillstrom Iterations---
---Mayo PBC Iterations---
---CMF Microfinance Iterations---


# Evaluation and Variance Table

In [29]:
# # Qini (regularize=False)
# df_model_eval = eval_table(model_eval_dict, variances=True, annotate_vars=True)

# df_model_eval

In [30]:
# Qini (regularize=True)
df_model_eval = eval_table(model_eval_dict, variances=True, annotate_vars=True)

df_model_eval

,TwoModel,InteractionTerm,BinaryTransformation,QuaternaryTransformation
Hillstrom,7.7943 ± 14.6244***,7.3166 ± 14.4166***,-11.5832 ± 12.4604***,-12.1184 ± 11.8034***
Mayo PBC,0.8755 ± 0.3517,0.2991 ± 0.4607,0.0394 ± 0.5006,1.1483 ± 0.5239
CMF Microfinance,13.7673 ± 7.4323**,13.3662 ± 5.4682**,NaN,NaN


In [31]:
print(df_model_eval.to_latex())
df_model_eval.to_latex("outputs_latex/df_model_eval.tex")

\begin{tabular}{lllll}
\toprule
{} &             TwoModel &      InteractionTerm &   BinaryTransformation & QuaternaryTransformation \\
\midrule
Hillstrom        &  7.7943 ± 14.6244*** &  7.3166 ± 14.4166*** &  -11.5832 ± 12.4604*** &    -12.1184 ± 11.8034*** \\
Mayo PBC         &      0.8755 ± 0.3517 &      0.2991 ± 0.4607 &        0.0394 ± 0.5006 &          1.1483 ± 0.5239 \\
CMF Microfinance &   13.7673 ± 7.4323** &   13.3662 ± 5.4682** &                    NaN &                      NaN \\
\bottomrule
\end{tabular}



In [34]:
import tabulate

print(
    tabulate.tabulate(
        tabular_data=df_model_eval.values,
        headers=df_model_eval.columns,
        tablefmt="pipe",
    )
)

| TwoModel            | InteractionTerm     | BinaryTransformation   | QuaternaryTransformation   |
|:--------------------|:--------------------|:-----------------------|:---------------------------|
| 7.7943 ± 14.6244*** | 7.3166 ± 14.4166*** | -11.5832 ± 12.4604***  | -12.1184 ± 11.8034***      |
| 0.8755 ± 0.3517     | 0.2991 ± 0.4607     | 0.0394 ± 0.5006        | 1.1483 ± 0.5239            |
| 13.7673 ± 7.4323**  | 13.3662 ± 5.4682**  | nan                    | nan                        |


# Summary